In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function linefragments(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            for k in sigma[h]
                line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[k], β)
                    end
                end
            end
        end
    end
    fragparams = []
    for line in params
        push!(line, 0.0, 1.0)
        line = sort(collect(Set(line)))
        push!(fragparams, line)
    end
    return fragparams
end

linefragments (generic function with 1 method)

## Dipendenze della funzione

In [3]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
             return nothing
        else
             # segments parallel: no intersection
             return nothing
        end
    end
    return α,β
end

intersection (generic function with 1 method)

## 0) Benchmark vecchia funzione 

In [11]:
V  =[0.0 2.0 2.0 -2.0 4.0 4.0 6.0 2.0 -1.0; 
      0.0 2.0 0.0  2.0 4.0 6.0 6.0 6.0  3.0]
EV = [[1, 2], [3, 4], [5, 6], [7, 6], [8, 9]]
spaceindex = [[2], [1], [4], [3], Int64[]]
@btime linefragments(V,EV,spaceindex)

  11.332 μs (248 allocations: 11.78 KiB)


5-element Array{Any,1}:
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]

## 1) Controllo se la funzione è type unstable

In [5]:
@code_warntype linefragments(V,EV,spaceindex)

Variables
  #self#::Core.Compiler.Const(linefragments, false)
  V::Array{Float64,2}
  EV::Array{Array{Int64,1},1}
  Sigma::Array{Array{Int64,1},1}
  m::Int64
  sigma::Array{Array{Int64,1},1}
  reducedsigma::Array{Array{Int64,1},1}
  params::Array{Array{Float64,1},1}
  @_9::Union{Nothing, Tuple{Int64,Int64}}
  fragparams::Array{Any,1}
  @_11::Union{Nothing, Tuple{Array{Float64,1},Int64}}
  @_12::Array{Array{Float64,1},1}
  @_13::Int64
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  h::Int64
  line1::Array{Float64,2}
  @_18::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_20::Int64
  line2::Array{Float64,2}
  out::Union{Nothing, Tuple{Any,Any}}
  α::Any
  β::Any
  line::Array{Float64,1}
  @_26::Any
  @_27::Any
  @_28::Any

Body::Array{Any,1}
1 ──        Core.NewvarNode(:(params))
│           Core.NewvarNode(:(@_9))
│           Core.NewvarNode(:(fragparams))
│           Core.NewvarNode(:(@_11))
│           (m = Main.length(Sigma))
│           (sigma = Main.map(Main.sort, Sigma))

│    %127 = (@_11 === nothing)::Bool
│    %128 = Base.not_int(%127)::Bool
└───        goto #30 if not %128
29 ─        goto #28
30 ┄        return fragparams


la funzione non  type unstable in quanto ho nell'output la stringa:

    Body::Array{Any,1}

In [12]:
function linefragments(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    @simd for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            @simd for k in sigma[h]
                line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[k], β)
                    end
                end
            end
        end
    end
    fragparams = []
    @simd for line in params
        push!(line, 0.0, 1.0)
        line = sort(collect(Set(line)))
        push!(fragparams, line)
    end
    return fragparams
end

V  =[0.0 2.0 2.0 -2.0 4.0 4.0 6.0 2.0 -1.0; 
      0.0 2.0 0.0  2.0 4.0 6.0 6.0 6.0  3.0]
EV = [[1, 2], [3, 4], [5, 6], [7, 6], [8, 9]]
spaceindex = [[2], [1], [4], [3], Int64[]]
@code_llvm linefragments(V,EV,spaceindex)


;  @ In[12]:1 within `linefragments'
define nonnull %jl_value_t* @japi1_linefragments_1771(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t*, i32 3
  %gcframe = alloca %jl_value_t*, i32 14, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 112, i1 false)
  %5 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 6
  %6 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 5
  %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 4
  %8 = bitcast %jl_value_t** %7 to { %jl_value_t* }*
  %9 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 3
  %10 = bitcast %jl_value_t** %9 to [1 x %jl_value_t*]*
  %11 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 2
  %12 = bitcast %jl_value_t** %11 to [1 x %jl_value_t*]*
  %13 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %13, align 8
  %14 = alloca [1 x i8], align 1
  %15 = alloca [1 x [1 x i64]], 